# Dimensionality in SuperCollider

After we took a look at the math perspective of vectors and matrices we now want to shift and expand our view on the topic by taking a look at how to create and interact with vectors and matrices in SuperCollider.

## From scalar to vector to matrix

### Scalar

As we discussed a scalar is a single numeric value.
This can either be a natural number or a real number.

In [1]:
5

-> 5

In [2]:
1.5

-> 1.5

In [3]:
sqrt(2.0)

-> 1.4142135623731

When working with rational numbers we must take into accont that a computer is [bound to a limited representation of our number](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) which leads sometimes to unexpected results.

In [4]:
(0.7/0.1).floor.postln;
7.0.floor.postln;

6.0
7.0
-> 7.0

Althogh $\left \lfloor \frac{0.7}{0.1} \right \rfloor = \lfloor 7 \rfloor = 7$, where $\lfloor x \rfloor$ is the [flooring](https://en.wikipedia.org/wiki/Floor_and_ceiling_functions) operator on the variable $x$, SuperCollider decides to return a 6 instead which is caused by the limited floating point precission.

### Vector

We can store multiple scalar values into an [Array](https://doc.sccode.org/Classes/Array.html).
Have in mind that [indexing of arrays start at 0 in most programming languages](https://www.cs.utexas.edu/users/EWD/transcriptions/EWD08xx/EWD831.html) where in math the convention is to start indexing from 1.

In [5]:
v = [2, 3];
v;

-> [ 2, 3 ]

We can scale each element of our array with a scalar by multiplying a scalar with our array `v`.

In [6]:
v*4

-> [ 8, 12 ]

In [7]:
v/4

-> [ 0.5, 0.75 ]

We can also multiply the array with another array.

In [8]:
[2,3]*[4,9]

-> [ 8, 27 ]

Lets see what happens when we multiply non-matching dimensions of our arrays.

In [9]:
[2,3]*[4,5,6]

-> [ 8, 15, 12 ]

A bit unexpecting that we do not receive an error - SuperCollider expands arrays in such operations to the longest array and cycles through the shorter ones. This allows for very concise code, but you have to take care about the input arrays yourself.
Lets see how this cascades when we want to multiply 3 arrays with different size.

In [10]:
[2, 3] * [4, 5, 6] * [7, 8, 9, 10]

-> [ 56, 120, 108, 80 ]

The results are calculated as following:

$$
\begin{aligned}
2 \cdot 4 \cdot 7 &= 7\\
3 \cdot 5 \cdot 8 &= 120\\
2 \cdot 6 \cdot 9 &= 108\\
3 \cdot 4 \cdot 10 &= 80
\end{aligned}
$$

### Matrix

Now we can also take a look at how a matrix is represented in SuperCollider.

In [11]:
n = [
    [2, 3, 4],
    [4, 5, 6],
]

-> [ [ 2, 3, 4 ], [ 4, 5, 6 ] ]

So it is basically just an array of arrays - but lets take a closer look at this as it is important to get the indices and the dimensions properly when working with matrices.

Lets say we want to represent a $3 \times 2$ matrix 
$$
\text{M} =
\begin{pmatrix}
   1 & 2\\
   3 & 4\\
   5 & 6
\end{pmatrix}
$$

In [12]:
m = [
    [1, 2],
    [3, 4],
    [5, 6],
];
m.postln;
m[0][1];

[ [ 1, 2 ], [ 3, 4 ], [ 5, 6 ] ]
-> 2

We see that the first dimension in a SuperCollider array are the rows, the 2nd dimension (the content of the arrays) are the columns - this fits nicely with our mathematical notation.

So lets try common operations such as the scaling of a matrix with a scalar.

In [13]:
m*2;

-> [ [ 2, 4 ], [ 6, 8 ], [ 10, 12 ] ]

The multiplication of a matrix with a scalar.

In [14]:
m + 1

-> [ [ 2, 3 ], [ 4, 5 ], [ 6, 7 ] ]

The multiplication of our a matrices with vectors.

In [15]:
m * [1, 0, 2]

-> [ [ 1, 2 ], [ 0, 0 ], [ 10, 12 ] ]

In [16]:
m*m

-> [ [ 1, 4 ], [ 9, 16 ], [ 25, 36 ] ]

In [17]:
m*n

-> [ [ 2, 6, 4 ], [ 12, 20, 18 ], [ 10, 18, 20 ] ]

Although the dimensions are matching with our mathematical definition the results do not match up.

### Sets

The way to declare a set in SuperCollider is straightforward.

In [18]:
a = Set[1, 2, 3, 1];
a;

-> Set[ 3, 2, 1 ]

Notice that although we declared duplicates in the set this duplication is ignored by SuperCollider as a set can only have unique elements.
We can also quickly take a look at the cardinality of sets.

In [19]:
a.size;

-> 3

Note also that due the circumstance that a set has no order we can also not access the *i*-th element of a set.

In [20]:
a[0]

ERROR: Message 'at' not understood.
RECEIVER:
Instance of Set {    (0x7ff488b78598, gc=C0, fmt=00, flg=00, set=02)
  instance variables [2]
    array : instance of Array (0x7ff47102d598, size=8, set=3)
    size : Integer 3
}
ARGS:
   Integer 0

PROTECTED CALL STACK:
	Meta_MethodError:new	0x7ff460514080
		arg this = DoesNotUnderstandError
		arg what = nil
		arg receiver = Set[ 3, 2, 1 ]
	Meta_DoesNotUnderstandError:new	0x7ff460516040
		arg this = DoesNotUnderstandError
		arg receiver = Set[ 3, 2, 1 ]
		arg selector = at
		arg args = [ 0 ]
	Object:doesNotUnderstand	0x7ff48810b200
		arg this = Set[ 3, 2, 1 ]
		arg selector = at
		arg args = nil
	a FunctionDef	0x7ff470edbf08
		sourceCode = "{ var result; \"**** JUPYTER ****\".postln; result = {a[0]}.value(); postf(\"-> %
\", result); \"\".postln;}"


## Working with matrices

Lets start by implementing a function which will multiply two matrices which also checks if this is a valid operation.

In [21]:
~matrixMul = { |a, b|
    if(a[0].size != b.size) { Error("matrix dimensions do not match").throw };
    if(a.every({|i| a[0].size == i.size}).not) {Error("input A has non identical dimensions").throw};
    if(b.every({|i| b[0].size == i.size}).not) {Error("input B has non identical dimensions").throw};
    a.collect { |d| (b * d).sum }
};

-> a Function

In [22]:
~matrixMul.(
    [[1, 1, 1], [1,1,3]],
    [[2],[1],[1]]
)

-> [ [ 4 ], [ 6 ] ]

We want to use this function to calculate downmixes.

To do this we first need to start the server.

In [23]:
s.boot;

-> localhost

In [24]:
Ndef(\x, {
    var a, b, mix, selected, gater;

    gater = {
        Trig1.kr(
            in: PulseDivider.kr(
                trig: Impulse.kr(8), 
                div: IRand(4, 9),
                start: IRand(0, 8)),
            dur: 1/8
        )
    };

    a = [
        [SinOsc.ar(freq: [440, 226, 782]) * gater.value],
        [Saw.ar(freq: [45, 881, 9000]) * gater.value],
        [Pulse.ar(freq: [8000, 1000, 500]) * gater.value],
        [Blip.ar(freq: [3, 8, 20], numharm: 140) * gater.value],
    ];

    // calculate a mixdown
    b = { [1.0, 1.0, 1.0, 1.0].rand } ! 1;
    //b = { LFNoise1.kr(1 ! 3).max(0) } ! 8;
    mix = ~matrixMul.(a, b);
    
    // flip through the signals
    selected = SelectX.ar(
        which: MouseX.kr(0, 7).round.lag, 
        array: mix
    );
    
    // each row is three-channel, mix to stereo
    Splay.ar(inArray: selected) * 0.1
}).play;

-> Ndef('x')

In Jupyter Notebook, we stop the audio playback of the SuperCollider server by executing a single dot.

In [25]:
.

-> CmdPeriod